# Test du code pyomo

Nous allons tester notre code pyomo sur une instance dont nous connaissons déjà la réponse

In [1]:
import pyomo.environ as pyo
import numpy as np
from itertools import product


## Construction de l'instance

Nous allons travailler sur l'instance avec les paramètres suivants : 

- 4 instants : $T$=[0, 1, 2]

- 2 zones : $A$ = ['A1', 'A2']

- 2 opérateurs; le notre 'I' et un autre opérateur concurrent qu'on va nommer 'J' donc $I$= ['I', 'J']

- les offres seront les suivants : 
    > -offre de 'I' : $O_I$ = ['$3G_I$', '$4G_I$']

    > -offre de 'J' : $O_J$ = ['$3G_J$', '$4G_J$']

    Donc O = ['$3G_I$', '$4G_I$', '$3G_J$', '$4G_J$'] et la nouvelle génération est la 4G.

- 2 sites : $S$ = ['S1', 'S2']

- chaque site Si recouvre la zone Ai où i $\in$ [1, 2]

- Initialement, chaque opérateur avait des utilisateurs initiaux comme le tableau qui suit :

| Zone | Opérateur | Technologie | Nombre de clients |
|------|-----------|-------------|--------------------|
| A1   | I         | $3G_I$     | 1000               |
| A1   | J         | $3G_J$     | 1000               |
| A1   | I         | $4G_I$     | 0                  |
| A1   | J         | $4G_J$     | 0                  |
| A2   | I         | $3G_I$     | 2000               |
| A2   | J         | $3G_J$     | 2000               |
| A2   | I         | $4G_I$     | 0                  |
| A2   | J         | $4G_J$     | 0                  |


- Zmax = 1 est constant :  à chaque instant on ne peut déployer la 4G que que sur un seul site

- $QA^t$ = 0 : il n'y a pas de contrainte gouvernementale

- On va choisir $CAPA_{NG}$ = $D_{NG}$ = 0 pour ne pas avoir de contraintes en plus

- Définition de $R_{comp}$ qui ne contient que les décisions de l'opérateur J qui ne déploie pas du tout la 4G:

| Zone | Instant (t) | $R_{comp}$ |
|------|-----------|-----------------|
| A1   | 0         | 0               |
| A1   | 1         | 0               |
| A1   | 2         | 0               |
| A2   | 0         | 0               |
| A2   | 1         | 0               |
| A2   | 2         | 0               |


- La fonction f va être défini comme suit : la moitié des clients qui ont la 3G chez l'un des deux opérateurs vont migrer vers la 4G de notre opérateur une fois à chaque instant où elle est déployé.


## Définition des paramètres

In [2]:
model = pyo.ConcreteModel()

# 1. SETS

T=[0, 1, 2]  # horizon temporel
A=['A1', 'A2']  # zones
I=['I', 'J']  # opérateurs
τ='I'  # notre opérateur
O=['3G_I', '3G_J', '4G_I', '4G_J']  # offres
O_i = {'I': ['3G_I', '4G_I'], 'J': ['3G_J', '4G_J']}  # offres par opérateur
NG = '4G_I'  # nouvelle génération
Si = ['S1', 'S2']  # sites de l’opérateur τ

model.T = pyo.Set(initialize=T)  # horizon temporel
model.A = pyo.Set(initialize=A)  # zones
model.I = pyo.Set(initialize=I)  # opérateurs
model.O = pyo.Set(initialize=O)  # offres !!! il faut mettre l'offre de chaque opérateur
model.O_i = O_i  # offres par opérateur
model.S = pyo.Set(initialize=Si) # sites de l’opérateur i

# Couples utiles
C_space = list(product([0,1], repeat=len(I)))  # Toutes les combinaisons de couverture
Sa_dict = {  # mapping a → {sites}
    'A1': ['S1'],
    'A2': ['S2']}
As_dict = {  # mapping s → {zones}
    'S1': ['A1'],
    'S2': ['A2']}


model.Cvec = pyo.Set(initialize=C_space)   # Toutes les combinaisons de couverture
model.Sa = Sa_dict                         # mapping a → {sites}
model.As = As_dict                         # mapping s → {zones}

# 2. PARAMÈTRES

Zmax_data = {0: 1, 1: 1, 2: 1}
model.Zmax = pyo.Param(model.T, initialize = Zmax_data)  # nombre max de sites déployables par période

QA_data = {0: 0, 1: 0, 2: 0}
model.QA = pyo.Param(model.T, initialize = QA_data) # couverture minimale de la population par période

ua0_data = {('A1', 'I', '3G_I'): 1000,
            ('A1', 'J', '3G_J'): 1000,
            ('A1', 'I', '4G_I'): 0,
            ('A1', 'J', '4G_J'): 0,
            ('A2', 'I', '3G_I'): 2000,
            ('A2', 'J', '3G_J'): 2000,
            ('A2', 'I', '4G_I'): 0,
            ('A2', 'J', '4G_J'): 0}


model.ua0 = pyo.Param(model.A, model.I, model.O, initialize=ua0_data)  # utilisateurs initiaux

DNG_data = {0: 0, 1: 0, 2: 0}
model.DNG = pyo.Param(model.T, initialize = DNG_data)  # DNG dépend du temps

CAPANG_data = {0: 0, 1: 0, 2: 0}
model.CAPANG = pyo.Param(model.T, initialize = CAPANG_data) # DNG et CAPANG dépendent du temps

u_a_data = {'A1': 2000, 'A2': 4000}
model.u_a = pyo.Param(model.A, initialize = u_a_data)  # utilisateurs totaux dans la zone a

Rcomp_data = {}
for t in model.T:
    for a in model.A:
        Rcomp_data[(t, a,'J')] =  0 # L'opérateur J déploie pas la 4G
model.Rcomp = pyo.Param(model.T, model.A, model.I, initialize=Rcomp_data, default=1) 

f_data = {}
for a in model.A:
    for C in model.Cvec:
        for o1 in model.O:
            for o2 in model.O:
                if C[0] == 1 and o2 == '4G_I' and o1 != '4G_I': # taux de migration si notre opérateur couvre la zone
                    f_data[(a, C, o1, o2)] = 0.5  # taux de migration si notre opérateur couvre la zone
                elif C[0] == 1 and o1 == '4G_I' and o2 == '4G_I': 
                    f_data[(a, C, o1, o2)] = 1 
                elif C[0] == 1 and o1 == '3G_I' and o2 == '3G_I': 
                    f_data[(a, C, o1, o2)] = 0.5
                elif C[0] == 1 and o1 == '3G_J' and o2 == '3G_J': 
                    f_data[(a, C, o1, o2)] = 0.5
                elif C[0] == 0 and o1 == '3G_I' and o2 == '3G_I': 
                    f_data[(a, C, o1, o2)] = 1
                elif C[0] == 0 and o1 == '3G_J' and o2 == '3G_J': 
                    f_data[(a, C, o1, o2)] = 1
                else:
                    f_data[(a, C, o1, o2)] = 0  # taux de migration sinon
model.f = pyo.Param(model.A, model.Cvec, model.O, model.O, initialize=f_data)  # taux de migration

## Définition des variables

In [3]:
# 3. VARIABLES

model.z = pyo.Var(model.T, model.S, within=pyo.Binary)
model.r = pyo.Var(model.T, model.A, within=pyo.Binary)
model.delta = pyo.Var(model.T, model.A, model.Cvec, within=pyo.Binary)

model.u = pyo.Var(model.T, model.A, model.I, model.O, within=pyo.NonNegativeReals)
model.u_site = pyo.Var(model.T, model.A, model.S, within=pyo.NonNegativeReals)  # on ne définit pas l'opérateur et l'offre car on ne parle que de notre opérateur et la NG

## Définition des contraintes

In [4]:
# 4. CONTRAINTES

# (2) r_ta ≤ sum_s z_ts
def coverage_upper(m, t, a):
    return m.r[t, a] <= sum(m.z[t, s] for s in Sa_dict[a])
model.c_2 = pyo.Constraint(model.T, model.A, rule=coverage_upper)

# (3) z_ts ≤ r_ta  ∀ s couvrant a
def coverage_lower(m, t, s, a):
    if a in As_dict[s]:
        return m.z[t, s] <= m.r[t, a]
    return pyo.Constraint.Skip
model.c_3 = pyo.Constraint(model.T, model.S, model.A, rule=coverage_lower)


def delta_implication(m, t, a, *C):
    # C est un tuple binaire représentant (cτ, c1, c2, ...)
    res = 1
    # opérateur τ (index 0)
    cτ = C[0]
    res *= (m.r[t, a] * cτ + (1 - cτ) * (1 - m.r[t, a]))
    # autres opérateurs
    autres_operateurs = list(m.I)[1:]  # on exclut τ
    for k, i in enumerate(autres_operateurs):
        ck = C[k+1]
        R = m.Rcomp[t, a, i]
        res *= (R * ck + (1 - ck) * (1 - R))

    return m.delta[t, a, C] == res 

model.c_4 = pyo.Constraint(model.T, model.A, model.Cvec, rule=delta_implication)

# (5) Migration non-linéaire
# --- Ajout des variables auxiliaires et paramètres M ---
# Note: suppose que model.ua0 est disponible comme borne supérieure pour u (tu l'as déjà)
# On crée un param M[a,C,o]
M = {}
for a in model.A:
    for C in model.Cvec:
        for o in model.O:
            M_val = 0.0
            for i_prev in model.I:
                for o_prev in model.O_i[i_prev]:
                    M_val += pyo.value(model.f[a, C, o_prev, o]) * pyo.value(model.u_a[a])
            M[(a, C, o)] = M_val

M_data = M

model.M = pyo.Param(model.A, model.Cvec, model.O, initialize=M_data, mutable=True)

# variable auxiliaire y_{t-1,a,C,o}
model.y = pyo.Var(model.T, model.A, model.Cvec, model.O, within=pyo.NonNegativeReals)

# (5) 1) y <= M * delta
def y_le_Mdelta(m, t, a, o, *C):
    if t == min(m.T): return pyo.Constraint.Skip
    return m.y[t-1, a, C, o] <= m.M[a, C, o] * m.delta[t-1, a, C]
model.c_y1 = pyo.Constraint(model.T, model.A, model.O, model.Cvec, rule=y_le_Mdelta)

# (5) 2) y >= 0  (déjà forcé par var NonNegativeReals) -> pas nécessaire

# (5) 3) y <= S
def y_le_S(m, t, a, o, *C):
    if t == min(m.T): return pyo.Constraint.Skip
    S_expr = sum(model.f[a, C, o_prev, o] * m.u[t-1, a, i_prev, o_prev]
                 for i_prev in m.I for o_prev in m.O_i[i_prev])
    return m.y[t-1, a, C, o] <= S_expr
model.c_y2 = pyo.Constraint(model.T, model.A, model.O, model.Cvec, rule=y_le_S)

# (5) 4) y >= S - M*(1-delta)
def y_ge_S_minus_M_1minusdelta(m, t, a, o, *C):
    if t == min(m.T): return pyo.Constraint.Skip
    S_expr = sum(model.f[a, C, o_prev, o] * m.u[t-1, a, i_prev, o_prev]
                 for i_prev in m.I for o_prev in m.O_i[i_prev])
    return m.y[t-1, a, C, o] >= S_expr - m.M[a, C, o] * (1 - m.delta[t-1, a, C])
model.c_y3 = pyo.Constraint(model.T, model.A, model.O, model.Cvec, rule=y_ge_S_minus_M_1minusdelta)

# Remplacer la contrainte migration non-linéaire par la somme des y
def migration_lin(m, t, a, i, o):
    if o in model.O_i[i]:    
        if t == min(m.T):
            return m.u[t, a, i, o] == pyo.value(m.ua0[a, i, o])
        return m.u[t, a, i, o] == sum(m.y[t-1, a, C, o] for C in m.Cvec)
    else:
        return m.u[t, a, i, o] == 0
model.c_5_lin = pyo.Constraint(model.T, model.A, model.I, model.O, rule=migration_lin)


# (6) u_NO = somme sur les sites # à revoir
def assign_users(m, t, a):
    return m.u[t, a, τ, NG] == sum(m.u_site[t, a, s] for s in Sa_dict[a])
model.c_6 = pyo.Constraint(model.T, model.A, rule=assign_users)

# # (7) capacité
def capacity(m, t, s):
    return sum(model.DNG[t] * m.u_site[t, a, s] for a in As_dict[s]) <= model.CAPANG[t] * m.z[t, s]
model.c_7 = pyo.Constraint(model.T, model.S, rule=capacity)

# (8) budget sur le nombre de sites déployés par période
def limit_z(m, t):
    if t == min(m.T):
        return sum(m.z[t, s] for s in model.S) <= model.Zmax[t]
    return sum(m.z[t, s] - m.z[t-1, s] for s in model.S) <= model.Zmax[t]

model.c_8 = pyo.Constraint(model.T, rule=limit_z)

# (9) Couverture population
def cov_pop(m, t):
    return sum(m.u_a[a] * m.r[t, a] for a in model.A) >= model.QA[t] * sum(m.u_a[a] for a in model.A)
model.c_9 = pyo.Constraint(model.T, rule=cov_pop)

## Définition d'objectif

In [5]:
# 5. OBJECTIF

def objective(m):
    T_end = max(m.T)
    return sum(m.u[T_end, a, τ, NG] for a in m.A)
model.obj = pyo.Objective(rule=objective, sense=pyo.maximize)

## Résoudre le problème

In [6]:
# Create solver
solver = pyo.SolverFactory('glpk')

# Solve
results = solver.solve(model, tee=True)

# Display solver status
print("\nSolver status:", results.solver.status)
print("Termination condition:", results.solver.termination_condition)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\DELL\AppData\Local\Temp\tmp1o8_xj1t.glpk.raw --wglp C:\Users\DELL\AppData\Local\Temp\tmpuik_j9xw.glpk.glp
 --cpxlp C:\Users\DELL\AppData\Local\Temp\tmpafi8bbyh.pyomo.lp
Reading problem data from 'C:\Users\DELL\AppData\Local\Temp\tmpafi8bbyh.pyomo.lp'...
C:\Users\DELL\AppData\Local\Temp\tmpafi8bbyh.pyomo.lp:1655: warning: lower bound of variable 'x5' redefined
C:\Users\DELL\AppData\Local\Temp\tmpafi8bbyh.pyomo.lp:1655: warning: upper bound of variable 'x5' redefined
294 rows, 155 columns, 600 non-zeros
36 integer variables, all of which are binary
1691 lines were read
Writing problem data to 'C:\Users\DELL\AppData\Local\Temp\tmpuik_j9xw.glpk.glp'...
1399 lines were written
GLPK Integer Optimizer, v4.65
294 rows, 155 columns, 600 non-zeros
36 integer variables, all of which are binary
Preprocessing...
2 hidden covering inequaliti(es) were detected
24 constraint coefficient(s) were reduced
83 r

## Affichage des résultats

In [7]:
print("--- Statut du Solveur ---")
print(f"Statut : {results.solver.status}")
print(f"Condition de Terminaison : {results.solver.termination_condition}")
print(f"Valeur de l'Objectif : {model.obj()}") # Afficher directement l'objectif
model.write('model.lp', io_options={'symbolic_solver_labels': True})

--- Statut du Solveur ---
Statut : ok
Condition de Terminaison : optimal
Valeur de l'Objectif : 4000.0


('model.lp', 2254832014624)

### Affichage des variables

In [9]:
# ---- AFFICHAGE DES SOLUTIONS ----

def afficher_variable(var):
    print(f"\n=== Variable {var.name} ===")
    try:
        for index in var:
            val = pyo.value(var[index])
            if abs(val) > 1e-6:  # n'affiche que les valeurs non nulles
                print(f"{index} = {val}")
    except TypeError:
        # variable scalaire
        print(pyo.value(var))

print("\n====================")
print("SOLUTIONS OPTIMALES")
print("====================")

# Variables binaires
afficher_variable(model.z)
afficher_variable(model.r)

# Variables utilisateurs
afficher_variable(model.u)

# Variables linéarisées
#afficher_variable(model.y)


SOLUTIONS OPTIMALES

=== Variable z ===
(0, 'S2') = 1.0
(1, 'S1') = 1.0
(1, 'S2') = 1.0

=== Variable r ===
(0, 'A2') = 1.0
(1, 'A1') = 1.0
(1, 'A2') = 1.0

=== Variable u ===
(0, 'A1', 'I', '3G_I') = 1000.0
(0, 'A1', 'J', '3G_J') = 1000.0
(0, 'A2', 'I', '3G_I') = 2000.0
(0, 'A2', 'J', '3G_J') = 2000.0
(1, 'A1', 'I', '3G_I') = 1000.0
(1, 'A1', 'J', '3G_J') = 1000.0
(1, 'A2', 'I', '3G_I') = 1000.0
(1, 'A2', 'I', '4G_I') = 2000.0
(1, 'A2', 'J', '3G_J') = 1000.0
(2, 'A1', 'I', '3G_I') = 500.0
(2, 'A1', 'I', '4G_I') = 1000.0
(2, 'A1', 'J', '3G_J') = 500.0
(2, 'A2', 'I', '3G_I') = 500.0
(2, 'A2', 'I', '4G_I') = 3000.0
(2, 'A2', 'J', '3G_J') = 500.0
